In [1]:
import requests
import numpy as np
import pandas as pd

In [2]:
INPUT_URL = "https://adventofcode.com/2022/day/7/input"
with open("session.txt", "r") as fh:
    SESSION = fh.readlines()[0]

In [3]:
def get_input() -> str:
    return requests.get(INPUT_URL, cookies={"session": SESSION}).text

In [58]:
input = get_input()

In [52]:
input = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
"""

In [59]:
# Part 1

from dataclasses import dataclass
from typing import Callable


@dataclass
class Action:
    cmd: str
    argv: list[str]
    out: list[str]


class Node:
    name: str
    size: int = 0
    parent: "Dir" = None

    def __init__(self, name: str, size: int) -> None:
        self.name = name
        self.size = size

    def print(self, prefix: str = "") -> str:
        print(f"{prefix}{self.name}, {self.__class__} ({self.size})")


class File(Node):
    pass


class Dir(Node):
    def __init__(self, name: str) -> None:
        super().__init__(name, 0)
        self.sub_dir: dict[str, "Dir"] = {}
        self.sub_files: list[File] = []

    def print(self, prefix: str = "") -> str:
        super().print(prefix)
        for f in self.sub_files:
            f.print(prefix + " - ")
        for d in self.sub_dir.values():
            d.print(prefix + " - ")

    def add_child(self, child: Node):
        assert not child.parent
        child.parent = self
        if isinstance(child, File):
            # print(f"!! add file {child.name} to {self.name}")
            self.sub_files.append(child)
        elif isinstance(child, Dir):
            assert child.name not in self.sub_dir
            # print(f"!! add dir {child.name} to {self.name}")
            self.sub_dir[child.name] = child
        self._update_size(child.size)

    def _update_size(self, size: int):
        self.size += size
        if self.parent:
            self.parent._update_size(size)


def parse_input(input: str) -> list[Action]:
    out: list[Action] = []
    for l in input.splitlines():
        if l.startswith("$"):
            parts = l.split(" ")
            out.append(Action(cmd=parts[1], argv=parts[2:], out=[]))
        else:
            out[-1].out.append(l)

    return out


def build_fs(actions: list[Action]) -> Dir:
    root = Dir("/")
    pwd = root
    for a in actions:
        if a.cmd == "cd":
            target = a.argv[0]
            # print(f"move to {target}")
            if target == "/":
                pwd = root
                continue
            elif target == "..":
                pwd = pwd.parent
                continue
            else:
                pwd = pwd.sub_dir[target]
                continue
        elif a.cmd == "ls":
            children = parse_ls(a.out)
            for child in children:
                # print(f"add {child.name} to {pwd.name}")
                pwd.add_child(child)
            continue
        else:
            raise RuntimeError(a)

    return root


def parse_ls(input: list[str]) -> list[Node]:
    out: list[Node] = []
    for l in input:
        parts = l.split(" ")
        name = parts[1]
        if parts[0] == "dir":
            out.append(Dir(name))
        else:
            size = int(parts[0])
            out.append(File(name, size))
    return out


def walk(root: Dir, fn: Callable[[Dir], None]):
    fn(root)
    for d in root.sub_dir.values():
        walk(d, fn)


result = 0


def solve_fn(node: Dir):
    global result
    if node.size <= 100000:
        result += node.size


actions = parse_input(input)
root = build_fs(actions)

walk(root, solve_fn)
result

1449447

In [60]:
# Part 2
TOTAL_DISK = 70000000
REQUIRED = 30000000

unused = TOTAL_DISK - root.size
to_delete = REQUIRED - unused

result = float("inf")


def solve_fn(node: Dir):
    global result
    if node.size >= to_delete:
        result = min(result, node.size)


walk(root, solve_fn)
result

8679207